In [21]:
import config
import spotipy as sp
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import pickle
from IPython.display import display, IFrame, clear_output

sp = sp.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

#uploading the dataset with our cluster column
audio_features = pd.read_csv("audio_features.csv")

#function to load the scaler and K-means pickles
def load(filename = "filename.pickle"): 
    try: 
        with open(filename, "rb") as f: 
            return pickle.load(f) 
        
    except FileNotFoundError: 
        print("File not found!")


# function to get user input and search for songs
def search_song():
    song_input = str(input("Please enter the song with the name of artist/group that you like : "))
    results = sp.search(q=song_input, limit=20)
    return results

# function to display songs and get user feedback
def display_song(results):
    for i in range(0, 19):
        track_id = results['tracks']['items'][i]['id']
        clear_output(wait=True)
        display(IFrame(src="https://open.spotify.com/embed/track/"+track_id+"?utm_source=generator",
               width="320",
               height="80",
               frameborder="0",
               allowtransparency="true",
               allow="encrypted-media",
              ))
        good_song = str(input("Is this the song you're looking for? Answer yes or no: ")).lower()

        while good_song != "yes" and good_song != "no":
            good_song = input("Answer can be only yes or no: ").lower()

        if good_song == "yes":
            return track_id

    print("Sorry, we couldn't find the song you were looking for.")
    return None

# function to extract song features, scale them, and predict cluster
def predict_cluster(track_id):
    select_song_features = sp.audio_features(track_id)
    del select_song_features[0]['key']
    del select_song_features[0]['mode']
    del select_song_features[0]['type']
    del select_song_features[0]['uri']
    del select_song_features[0]['track_href']
    del select_song_features[0]['analysis_url']
    del select_song_features[0]['time_signature']
    
    # convert to dataframe
    song_df = pd.DataFrame(select_song_features, index=[0])
    
    # remove id for clustering
    song_df.drop(columns="id", inplace=True)
    
    # scale features
    scaler_new_song = load("Model/scaler_song.pickle")
    scaled_song_df = pd.DataFrame(scaler_new_song.transform(song_df), columns=song_df.columns)
    
    # predict cluster
    kmeans_new_song = load("Model/kmeans_10.pickle")
    cluster = kmeans_new_song.predict(scaled_song_df)[0]
    return cluster

# function to recommend a song from the predicted cluster
def recommend_song(cluster):
    #audio_features = pd.read_csv(audio_features.csv")
    random_from_cluster = audio_features[audio_features['cluster'] == cluster].sample()
    id_recommended = random_from_cluster['id'].values[0]
    return id_recommended

# function to display the recommended song
def display_recommended_song(id_recommended):
    display(IFrame(src="https://open.spotify.com/embed/track/"+id_recommended+"?utm_source=generator",
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      ))

# main function to run the entire program
def main():
    results = search_song()
    track_id = display_song(results)

    if track_id is not None:
        cluster = predict_cluster(track_id)
        id_recommended = recommend_song(cluster)
        display_recommended_song(id_recommended)

# run the program
main()

Is this the song you're looking for? Answer yes or no: yes
